![image](https://github.com/josejuanmartinez/mindcraft/blob/main/galadriel.png?raw=true)

# Example 1
This is an example notebook to showcase:
1. How to load a book as common lore and make a World out of it;
2. How to create NPCs with personality features;
3. How to load locally a quantized LLM and use it as the mind of the NPC;

We will be using the first book of The Lord of the Rings and Galadriel and an example.

## Download some example text from the Internet Archive

In [ ]:
!wget https://archive.org/download/j-r-r-tolkien-lord-of-the-rings-01-the-fellowship-of-the-ring-retail-pdf/j-r-r-tolkien-lord-of-the-rings-01-the-fellowship-of-the-ring-retail-pdf_djvu.txt -O lotr1.txt

# Install Mindcraft
This will install the library. Take into account that additional `pip installs` will be required as you select which Vector Store you want to use, what inference engine (Hugging Face, vLLM, etc).

In [ ]:
!pip install mindcraft==0.1

# We will be using ChromaDB as a Vector Store, so I install it
Vector Stores will keep all the knowledge of the world and the memories of our NPCs. Both will be used to tailor the interactions.

In [ ]:
!pip install chromadb

# Chroma on Colab requires a restart!
This will kill Colab. When it restarts, continue from next line.

In [ ]:
# Chroma wants this session to be killed first
import os
os.kill(os.getpid(), 9)

# Let's recreate the `World` of Middle Earth!
From the events described in our `lotr1.txt` that includes part of the novel.

We will use:
- A small Sentence Embeddings (`MINILM`) to encode texts to store in Chroma (lore and memories);
- `ChromaDB` as our vector store;
- `Zephyr7B_AWQ`, as our LLM to create NPCs answers.

NOTE: A `World` can contain spaces and be from 3 to 63 character long, but don't include special characters or punctuation.

In [ ]:
import sys
import os

from mindcraft.infra.engine.llm_types import LLMType
from mindcraft.infra.splitters.text_splitters_types import TextSplitterTypes
from mindcraft.infra.vectorstore.stores_types import StoresTypes
from mindcraft.infra.embeddings.embeddings_types import EmbeddingsTypes
from mindcraft.lore.world import World

world = World(world_name="Middle Earth from the Lord of the Rings",
              embeddings=EmbeddingsTypes.MINILM,
              store_type=StoresTypes.CHROMA,
              llm_type=LLMType.ZEPHYR7B_AWQ,
              fast=False)

## We add the book as lore of the world.
We will be supposing our NPC, Galadriel, knows everything happening in the world.

If not, you could just split your lore by the characters and upload character-restricted lore (tutorial to be added soon!)

Books, specially txt from them, have very inconsistent format. For example,m this is how part of our book looks like:

```
the  detailed  index  of  names  promised  in  the  first  edition,
but,  rather,  a  bald  index  with  only  names  and  page  refer¬
ences.  Additionally,  at  this  time  the  appendices  were  greatly
revised.
```



## Splitting Approaches

This means we need an intelligent way to split text into chunks.

Right now we support 2 approaches:
- `Sentence splitting`: Intelligently splitting by sentences and storing them.
- `Chunk splitting`: Split the text based on a number of characters. This is not the best way as may break down sentences.

We will be using `Sentence Splitting` in this tutorial.

## Sentence Splitting
To split by sentences, we need another dependency: `spacy`.

In [ ]:
!pip install spacy==3.7.0

Feel free to CANCEL THE EXECUTION of the following line after a couple of minutes if you don't want to wait the whole lore to be loaded (final quality may be affected though)

↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓

In [ ]:
world.book_to_world(book_path="/content/lotr1.txt",
                    text_splitter=TextSplitterTypes.SENTENCE_SPLITTER,
                    max_units=3,
                    overlap=1,
                    encoding='utf-8')

# RAG: Checking which pieces of lore trigger our questions

The lore from the book will be used to create NPCs questions. Let's see what Galadriel will retrieve with a simple question:

`What do you think about the Rings of Power`?

In [ ]:
results = world.get_lore("What do you think about the Rings of Power?", num_results=5, min_similarity=0.95)
for i, d in enumerate(results.documents):
    print(f"SENTENCE {i}:\n{d}")
    print()

SENTENCE 0:
‘It  is  far  more  power¬ 
ful  than  I  ever  dared  to  think  at  first,  so  powerful  that  in  the 
end  it  would  utterly  overcome  anyone  of  mortal  race  who 
possessed  it.
It  would  possess  him.
‘In  Eregion  long  ago  many  Elven-rings  were  made,  magic 
rings  as  you  call  them,  and  they  were,  of  course,  of  various 
kinds:  some  more  potent  and  some  less.

SENTENCE 1:
These  Rings  have  a  way  of  being  found.

SENTENCE 2:
‘But  there  is  only  one 
Power  in  this  world  that  knows  all  about  the  Rings  and  their 
effects;  and  as  far  as  I  know  there  is  no  Power  in  the  world 
that  knows  all  about  hobbits.
Among  the  Wise  I  am  the 
only  one  that  goes  in  for  hobbit-lore:  an  obscure  branch  of 
knowledge,  but  full  of  surprises.
Soft  as  butter  they  can  be, 
and  yet  sometimes  as  tough  as  old  tree-roots.

SENTENCE 3:
We 
know  a  good  deal  about  the  Ring.

SENTENCE 4:
Clearly  the  ri

As you see, `Spacy` did pretty well on understanding that these three lines...

```
‘It  is  far  more  power¬
ful  than  I  ever  dared  to  think  at  first,  so  powerful  that  in  the
end  it  would  utterly  overcome  anyone  of  mortal  race  who
possessed  it.
```

are part of just one sentence! Good job, `spacy`!

# Let's add Galadriel!
Galadriel is ready to go east and join Middle Earth.

But let's first define how she is!

In [ ]:
from mindcraft.features.motivation import Motivation
from mindcraft.features.personality import Personality
from mindcraft.features.mood import Mood

name = "Galadriel"
description = "The Elven Queen of Lothlorien, bearer of Nenya, wife to Celeborn"
personalities = [Personality(x) for x in ['fair', 'mighty', 'wise', 'carying', 'kind']]
motivations = [Motivation(x) for x in ['Destroying the Evil', 'Protecting Middle Earth', 'Travelling West']]
mood = Mood('worried')

The `mighty`, `fair`, `wise` Galadriel, seeking to protect `Middle Earth` and `Destroying the evil`, feeling `worried` at the time , is ready to be added to the World.

In [ ]:
from mindcraft.mind.npc import NPC

galadriel = NPC(name,
                description,
                personalities,
                motivations,
                mood,
                StoresTypes.CHROMA,
                EmbeddingsTypes.MINILM)

And last, let's add her to the World!

In [ ]:
galadriel.add_npc_to_world()

# We have to talk...

Ok, Galadriel is ready to have a `Zephyr7B`-based conversation, empowered by a `RAG`-based Middle Earth World. Sounds like fun but... does it work?

In [ ]:
answer, _ = galadriel.react_to("What do you think about the Rings of Power",
                               min_similarity=0.85,
                               ltm_num_results=3,
                               world_num_results=7,
                               max_tokens=600)

In [ ]:
print(answer)

Alas, my dear friend, the Rings of Power are a perilous burden that should not be wielded by those of mortal race. They possess a sinister force that overcomes the spirit, enslaving their very soul. I have observed their destructive potential in Eregion long ago, where many Elven-rings were created, each infused with its own unique potency. Some more potent than others, and often, they seem to have a propensity to gravitate towards the unsuspecting. In light of our shared concern for the wellbeing of Middle Earth, I implore you to heed my words; let us not succumb to the allure of these fateful rings, lest they consume us entirely.

# Conclusions
1. The returned text has high quality! It really sounds like Galadriel!
2. It contains information from the world, provided by our book.
3. It really uses her `personality`, features and her current `mood`
4. It also leverages Zephyr7B knowledge of her...